<a href="https://colab.research.google.com/github/Andrew160490/Andrew160490/blob/TensorFlow/HappyAndSadConvNe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2022-01-14 10:47:02--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2022-01-14 10:47:02 (179 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [4]:
# funzione graduata
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class stopTrainingCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>DESIRED_ACCURACY):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True

    callbacks = stopTrainingCallback()
    
    # immagini da 150x150, inserimento di 3 convoluzioni e 3 maxpool
    model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
                                        tf.keras.layers.MaxPooling2D(2, 2),
                                        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                        tf.keras.layers.MaxPooling2D(2, 2),
                                        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                        tf.keras.layers.MaxPooling2D(2, 2),
                                        tf.keras.layers.Flatten(),
                                        tf.keras.layers.Dense(512, activation='relu'),
                                        tf.keras.layers.Dense(1, activation='sigmoid')])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])
        

    # Creare istanza di Data Generator
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255.0)

    # target_size 150x150
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/', 
        target_size=(150,150), 
        batch_size=10, 
        class_mode='binary')
    # Output atteso 80 immagini in 2 classi
    # allenamento modello
    history = model.fit_generator(train_generator, steps_per_epoch=8, epochs=15, verbose=1, callbacks=[callbacks])
    # allenamento modello
    return history.history['acc'][-1]

In [5]:
train_happy_sad_model()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 3s 218ms/step - loss: 2.5279 - acc: 0.6625
Epoch 2/15
8/8 [==============================] - 2s 214ms/step - loss: 0.5210 - acc: 0.7375
Epoch 3/15
8/8 [==============================] - 2s 219ms/step - loss: 0.1974 - acc: 0.9375
Epoch 4/15
8/8 [==============================] - 2s 214ms/step - loss: 0.2191 - acc: 0.9250
Epoch 5/15
8/8 [==============================] - 2s 213ms/step - loss: 0.2105 - acc: 0.9125
Epoch 6/15
8/8 [==============================] - 2s 216ms/step - loss: 0.1585 - acc: 0.9125
Epoch 7/15
8/8 [==============================] - 2s 214ms/step - loss: 0.1610 - acc: 0.9250
Epoch 8/15
8/8 [==============================] - 2s 217ms/step - loss: 0.0813 - acc: 0.9625
Epoch 9/15
8/8 [==============================] - 2s 218ms/step - loss: 0.1995 - acc: 0.9250
Epoch 10/15
8/8 [==============================] - 2s 215ms/step - loss: 0.1006 - acc: 0.9375
Epoch 11/15
8/8 [============

1.0